In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from transformers import AutoTokenizer, AutoModel
from flask import Flask, render_template

In [7]:
import pandas as pd

file1_path = "E:/studies/project_ideatiaon/Mid Term/NDTV.csv"
file2_path = "E:/studies/project_ideatiaon/Mid Term/India Today.csv"

df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

df_combined = pd.concat([df1, df2], ignore_index=True)

combined_file_path = 'combined_data.csv'
df_combined.to_csv(combined_file_path, index=False)

df_combined_loaded = pd.read_csv(combined_file_path)

print(df_combined_loaded)


                                                Title  \
0   Delhi govt postpones odd-even plan as air qual...   
1       Parliament Winter Session to start from Dec 4   
2   Ministry of I&B approves Comprehensive “Digita...   
3   Addressing stubble burning: Indian Agricultura...   
4   Israel and Hamas negotiating for release of ci...   
5   UK’s ICE: Anusha Shah takes over as first Indi...   
6   Israel must protect Palestinians in West Bank ...   
7   इलेक्ट्रॉनिक वस्तुओं की गारंटी या वारंटी की अव...   
8   Defence minister directs swift release of thir...   
9   India's growth outlook soars: Fitch raises med...   
10  E-Summit 2023: NIT Raipur sparks innovation an...   
11  PAK vs ENG: Pakistan captain explains how they...   
12  ICC suspends Sri Lanka Cricket with immediate ...   
13  This day, that year: South Africa returns to i...   
14  'Are you doing anything?': Green Tribunal to P...   
15  76 years ago, Gandhi told India and Pakistan h...   
16  AUS vs BAN Live: BAN lose T

In [12]:
df_combined_loaded.head()

,Title,Article_Content
0,Delhi govt postpones odd-even plan as air qual...,“The AQI which was 450+ has now reached around...
1,Parliament Winter Session to start from Dec 4,NaN
2,Ministry of I&B approves Comprehensive “Digita...,NaN
3,Addressing stubble burning: Indian Agricultura...,NaN
4,Israel and Hamas negotiating for release of ci...,NaN


# TEXT PREOCESSING WITH TRANSFOMER BASED NLP

In [8]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import torch

# Load the pre-trained transformer model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Read the combined CSV file into a Pandas DataFrame
combined_file_path = 'combined_data.csv'
df_combined = pd.read_csv(combined_file_path)

# Tokenize and encode news text using the transformer model
embeddings = []

for article_content in df_combined['Article_Content']:
    article_content = str(article_content)
    
    # Tokenize and encode the text
    tokens = tokenizer(article_content, return_tensors='pt', max_length=512, truncation=True)
    
    with torch.no_grad():
        output = model(**tokens)
    
    # Extract embeddings for the [CLS] token (first token)
    cls_embedding = output['last_hidden_state'][:, 0, :].squeeze().detach().numpy()
    embeddings.append(cls_embedding)

# Adding the embeddings to the DataFrame
df_combined['embeddings'] = embeddings

# Saving the DataFrame with embeddings to a new CSV file
df_combined.to_csv('combined_data_with_embeddings.csv', index=False)

print(df_combined)


                                                Title  \
0   Delhi govt postpones odd-even plan as air qual...   
1       Parliament Winter Session to start from Dec 4   
2   Ministry of I&B approves Comprehensive “Digita...   
3   Addressing stubble burning: Indian Agricultura...   
4   Israel and Hamas negotiating for release of ci...   
5   UK’s ICE: Anusha Shah takes over as first Indi...   
6   Israel must protect Palestinians in West Bank ...   
7   इलेक्ट्रॉनिक वस्तुओं की गारंटी या वारंटी की अव...   
8   Defence minister directs swift release of thir...   
9   India's growth outlook soars: Fitch raises med...   
10  E-Summit 2023: NIT Raipur sparks innovation an...   
11  PAK vs ENG: Pakistan captain explains how they...   
12  ICC suspends Sri Lanka Cricket with immediate ...   
13  This day, that year: South Africa returns to i...   
14  'Are you doing anything?': Green Tribunal to P...   
15  76 years ago, Gandhi told India and Pakistan h...   
16  AUS vs BAN Live: BAN lose T

# Reinforcement Learning for Clustering

In [24]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Define the actor network
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, action_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim=-1)  # Use softmax to get a probability distribution
        return x

# Define the critic network
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 1)

    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the DDPG agent
class DDPGAgent:
    def __init__(self, state_dim, action_dim, actor_lr=1e-4, critic_lr=1e-3, gamma=0.99, tau=1e-2, buffer_size=100000):
        self.actor = Actor(state_dim, action_dim)
        self.actor_target = Actor(state_dim, action_dim)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=actor_lr)

        self.critic = Critic(state_dim, action_dim)
        self.critic_target = Critic(state_dim, action_dim)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_lr)

        self.gamma = gamma
        self.tau = tau

        # Initialize replay buffer
        self.buffer_size = buffer_size
        self.buffer = []

    def select_action(self, state):
        state = torch.FloatTensor(state)
        action = self.actor(state).detach().numpy().flatten()
        return action
    
    def add_noise(self, action):
        return action

    def remember(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
        if len(self.buffer) > self.buffer_size:
            self.buffer.pop(0)

    def update_model(self, batch_size):
        if len(self.buffer) < batch_size:
            return

        # Sample a random mini-batch from the replay buffer
        batch = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, next_states, dones = zip(*[self.buffer[i] for i in batch])
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)  # Convert actions to FloatTensor
        rewards = torch.FloatTensor(rewards).unsqueeze(1)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones).unsqueeze(1)

        # Update the critic
        next_actions = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, next_actions)
        Q_targets = rewards + self.gamma * Q_targets_next * (1 - dones)
        Q_expected = self.critic(states, actions)
        critic_loss = nn.MSELoss()(Q_expected, Q_targets)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Update the actor
        actions_pred = self.actor(states)
        actor_loss = -torch.mean(self.critic(states, actions_pred))

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks with soft update
        self.soft_update(self.critic, self.critic_target)
        self.soft_update(self.actor, self.actor_target)

    def soft_update(self, local_model, target_model):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)

# Initialize the DDPG agent
state_dim = df_combined['embeddings'][0].shape[0]  # Assuming embeddings have the same length
action_dim = num_clusters  # Number of clusters
agent = DDPGAgent(state_dim, action_dim)

# Training loop
num_episodes = 1000
max_steps = 200

for episode in range(num_episodes):
    state = random.randint(0, len(df_combined) - 1)  # start with a random state
    total_reward = 0

    for step in range(max_steps):
        action = agent.select_action(df_combined['embeddings'][state])
        next_state = random.randint(0, len(df_combined) - 1)  # simulate moving to the next state
        reward = np.dot(df_combined['embeddings'][state], df_combined['embeddings'][next_state])

        agent.remember(df_combined['embeddings'][state], action, reward, df_combined['embeddings'][next_state], False)
        agent.update_model(batch_size=64)

        state = next_state
        total_reward += reward

        if random.uniform(0, 1) < 0.1:  # a simple termination condition for illustration
            break

    print(f"Episode {episode + 1}, Total Reward: {total_reward}")


df_combined['optimal_clusters'] = df_combined['embeddings'].apply(lambda x: agent.select_action(x))
df_combined.to_csv('combined_data_with_optimal_clusters_ddpg.csv', index=False)

print(df_combined[['Title', 'Article_Content', 'optimal_clusters']])

Episode 1, Total Reward: 3406.2612838745117
Episode 2, Total Reward: 1166.8651885986328
Episode 3, Total Reward: 1327.8512420654297
Episode 4, Total Reward: 125.79322052001953
Episode 5, Total Reward: 2556.443977355957
Episode 6, Total Reward: 1173.6866607666016
Episode 7, Total Reward: 3298.960868835449
Episode 8, Total Reward: 605.8953399658203
Episode 9, Total Reward: 324.9254379272461
Episode 10, Total Reward: 3302.4556198120117
Episode 11, Total Reward: 1564.945671081543
Episode 12, Total Reward: 1927.007308959961
Episode 13, Total Reward: 2175.2229232788086
Episode 14, Total Reward: 3694.8954544067383
Episode 15, Total Reward: 1071.1548080444336
Episode 16, Total Reward: 8134.921661376953
Episode 17, Total Reward: 3715.805992126465
Episode 18, Total Reward: 828.6531829833984
Episode 19, Total Reward: 315.6412353515625
Episode 20, Total Reward: 2932.776206970215
Episode 21, Total Reward: 95.8471908569336
Episode 22, Total Reward: 2523.497901916504
Episode 23, Total Reward: 973.646

# News Categorization

In [31]:
df_combined['optimal_clusters'] = df_combined['optimal_clusters'].apply(lambda x: np.argmax(x))


def categorize_news(cluster):
    category_mapping = {
        0: 'General',
        1: 'Politics',
        2: 'Technology',
        3: 'Finance',
        4: 'Entertainment',
    }

    return category_mapping.get(cluster, 'Uncategorized')

df_combined['Category'] = df_combined['optimal_clusters'].apply(categorize_news)

print(df_combined[['Title', 'Article_Content', 'optimal_clusters', 'Category']])

                                                Title  \
0   Delhi govt postpones odd-even plan as air qual...   
1       Parliament Winter Session to start from Dec 4   
2   Ministry of I&B approves Comprehensive “Digita...   
3   Addressing stubble burning: Indian Agricultura...   
4   Israel and Hamas negotiating for release of ci...   
5   UK’s ICE: Anusha Shah takes over as first Indi...   
6   Israel must protect Palestinians in West Bank ...   
7   इलेक्ट्रॉनिक वस्तुओं की गारंटी या वारंटी की अव...   
8   Defence minister directs swift release of thir...   
9   India's growth outlook soars: Fitch raises med...   
10  E-Summit 2023: NIT Raipur sparks innovation an...   
11  PAK vs ENG: Pakistan captain explains how they...   
12  ICC suspends Sri Lanka Cricket with immediate ...   
13  This day, that year: South Africa returns to i...   
14  'Are you doing anything?': Green Tribunal to P...   
15  76 years ago, Gandhi told India and Pakistan h...   
16  AUS vs BAN Live: BAN lose T